# Upload Slit IDs to PosFidSpecID Map

In [2]:
import numpy as np
import pandas as pd

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe

### Need to download the P#_SC#_mappingFRDverified.csv for the Petal you are updating and type it in below

In [4]:
df = pd.DataFrame.from_csv('P2_SC0_mappingFRDverified.csv')

/Users/parkerf/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  if __name__ == '__main__':


In [5]:
#Get these as lists because they are easier to manipulate
sbf = np.array(df['slit block:fiber num'])
slit = np.array(df['Cable ID'])

In [6]:
# Split the slit block and fiber number and label them with 'B' and 'F'
sb = []
sf = []
for f in sbf:
    if f == 'DEAD':
        sb.append("None")
        sf.append("None")
    else:
        sb.append('B'+(f.split(":")[0]).zfill(2))
        sf.append('F'+(f.split(":")[1]).zfill(2))

In [7]:
df['SLITBLOCK_ID'] = sb
df['BLOCKFIBER_ID'] = sf

In [8]:
#Make the slit id into a single number
ss = [int(x[-1]) for x in slit]
df['SLIT_ID'] = ss

## Upload data to google sheet

In [9]:
data = df.to_records()

In [ ]:
## Update Petal No.
PETAL = 2

In [12]:
url = 'https://docs.google.com/spreadsheets/d/1lJ9GjhUUsK2SIvXeerpGW7664OFKQWAlPqpgxgevvl8/edit#gid=0'
credentials = 'google_access_account.json'
scope = ['https://spreadsheets.google.com/feeds', 'https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)
client = gspread.authorize(creds)
sheet = client.open_by_url(url).sheet1
df  = get_as_dataframe(sheet, parse_dates=True, usecols=[0,1,2,3,4,6,8,14], skiprows=20, header=0)
dd = df.to_records(index=False)
dd = dd[dd['PETAL_ID'] == int(PETAL)]
#map = df.loc[df['PETAL_ID'].isin([int(2)])]     

In [17]:
for d in data: 
    hole = float(d['DEVICE_LOC'])
    rowID = sheet.find(dd[dd['DEVICE_LOC']==hole]['FULL_SI_ID'][0]).row
    sheet.update_cell(rowID, 11, str(d['SLIT_ID']))
    sheet.update_cell(rowID, 12, d['SLITBLOCK_ID'])
    sheet.update_cell(rowID, 13, d['BLOCKFIBER_ID'])